In [ ]:
'''阈值逻辑单元(threshold logic unit)，感知器由一层TLU组成，只能解决线性可分类问题。当由多个TLU构成多层感知器(MLU，包含有隐层单元)
可以解决单个TLU的局限，反向传播算法中，使用Logistic函数作为激活函数σ(z) = 1 / (1 + exp(–z))代替了阶跃，这是必要的,因为阶跃函数只包
含平坦的段,因此没有梯度来工作(梯度下降不能在平面上移动),而 Logistic 函数到处都有一个定义良好的非零导数,允许梯度下降在每个步上取得一些
进展。反向传播算法可以与其他激活函数一起使用,而不是 Logistic 函数。另外两个流行的激活函数是:1.双曲正切函数tanh (z) = 2σ(2z) – 1
就像 Logistic 函数,它是 S 形的、连续的、可微的,但是它的输出值范围从-1到1(不是在 Logistic 函数的 0 到 1),这往往使每个层的输出在训练
开始时或多或少都正则化了(即以 0 为中心)。这常常有助于加快收敛速度。2.ReLU函数:ReLU (z) = max (0, z)。(大于0时相当于y=x)它是连续的,
但不幸的是在z=0时不可微(斜率突然改变,这可以使梯度下降反弹)。然而,在实践中,它工作得很好,并且具有快速计算的优点。最重要的是,它没有最大输
出值的事实也有助于减少梯度下降期间的一些问题(后期补充)'''
#加载数据
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import struct
path = os.path.join(os.getcwd(), 'datasets')
def load_mnist(path, kind):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels.idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images.idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)
 
    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)
 
    return images, labels
train_images, train_labels = load_mnist(path, 'train')
test_images, test_labels = load_mnist(path, 't10k')

In [ ]:
'''TensorFlow 程序通常分为两部分:第一部分构建计算图谱(这称为构造阶段),第二部分运行它(这是执行阶段)。'''
import tensorflow as tf
#构建
x = tf.Variable(3, name='x')  # 向图谱中创建一个变量初始值为3，名称为x
y = tf.Variable(4, name='y')
f = x ** 2 * y + y + 2 
#运行
with tf.Session() as sess:  # 创建一个会话
    x.initializer.run() # 初始化变量
    y.initializer.run()
    result = f.eval() # 求出y的值

# way2 也可以使用这种方法来初始化变量
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     init.run()
#     result = f.eval()

print(result)

In [ ]:
'''管理图谱，可同时存在多个图谱，可选择将变量存放于某一个图谱中(一般存放于默认图谱中)'''
x1 = tf.Variable(1)
graph = tf.Graph()  # 创建一个新的图谱
with graph.as_default():  # 将新创建的图谱 暂时 定义为默认图谱
    x2 = tf.Variable(2)  # 在默认图谱中创建x2变量
x2.graph is graph  # 返回True，说明x2位于graph图谱
x2.graph is tf.get_default_graph()  # 返回Flase,graph图谱并不是默认图谱，说明上述操作只是 暂时 定义graph为默认图谱
x1.graph is tf.get_default_graph()  # 返回True
#tf.reset_default_graph()  # 重置默认图

In [ ]:
'''变量的生命周期在会话创建时开始，在会话关闭时结束。'''
'''tf进行矩阵运算，先创建变量，在创建会话进行计算'''
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

'''constant 创建常量  transpose 矩阵转置 matric_inverse 矩阵求逆 '''
'''计算theta=(XT*X)-1 * X * y'''
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)),XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()
print(theta_value)

In [15]:
'''手动操作(不使用API)实现梯度下降'''
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-15-78e63d77156b>, line 7)

In [ ]:
help(tf.transpose)
#train_data